In [1]:
from qutip import *
from qutip.piqs import *
import numpy as np
import math
import matplotlib.pyplot as plt

np.set_printoptions(threshold=np.inf)
qutip.about()


QuTiP: Quantum Toolbox in Python
Copyright (c) QuTiP team 2011 and later.
Current admin team: Alexander Pitchford, Nathan Shammah, Shahnawaz Ahmed, Neill Lambert, Eric Giguère, Boxi Li, Jake Lishman, Simon Cross and Asier Galicia.
Board members: Daniel Burgarth, Robert Johansson, Anton F. Kockum, Franco Nori and Will Zeng.
Original developers: R. J. Johansson & P. D. Nation.
Previous lead developers: Chris Granade & A. Grimsmo.
Currently developed through wide collaboration. See https://github.com/qutip for details.

QuTiP Version:      4.7.1
Numpy Version:      1.22.0
Scipy Version:      1.7.3
Cython Version:     None
Matplotlib Version: 3.7.1
Python Version:     3.9.18
Number of CPUs:     24
BLAS Info:          OPENBLAS
OPENMP Installed:   False
INTEL MKL Ext:      False
Platform Info:      Linux (x86_64)
Installation path:  /home/harsh/.local/lib/python3.9/site-packages/qutip
Please cite QuTiP in your publication.
For your convenience a bibtex reference can be easily generated usin

In [2]:
# Coefficients of collapse operators in PI Basis
def bm(j,m):
    return (-np.sqrt((j+m)*(j+m-1)))

def dm(j,m):
    return (np.sqrt((j-m+1)*(j-m+2)))

def bp(j,m):
    return (np.sqrt((j-m)*(j-m-1)))

def dp(j,m):
    return (-np.sqrt((j+m+1)*(j+m+2)))

def bz(j,m):
    return (np.sqrt((j-m)*(j+m)))

def dz(j,m):
    return (np.sqrt((j+m+1)*(j-m+1)))

def cj(N,J):
    if J== N/2:
        return (1/(2*J))

    else:
        return ((1/(2*(J)))*((N/2+1)/(J+1)))

def cjp1(N,J):
    return ((1/(2*(J+1)))*((N/2-J)/(2*J+1)))

def cjm1(N,J):
    return ((1/(2*(J)))*((N/2+J+1)/(2*J+1)))

def rate(ch,p,q,N,J,M,dr=1,dt=1):
    r = 0;
    if p==-1:
        if q==-1:
            r = bm(J,M)
        elif q==0:
            r = bz(J,M)
        else:
            r = bp(J,M)
    elif p==0:
        if q==-1:
            r = am(J,M)
            #if ch=='l':
                #r *= cjm1(N,J)
        elif q==0:
            r = M
            #if ch=='l':
                #r *= cjm1(N,J)
        else:
            r = ap(J,M)
            #if ch=='l':
                #r *= cjm1(N,J)
    elif p==1:
        if q==-1:
            r = dm(J,M)
        elif q==0:
            r = dz(J,M)
        else:
            r = dp(J,M)

    else:
        # Continuous dephasing
        r = 1-1j*dt*(- (1j*dr[0]/2)*(N/2-M) - (1j*dr[2]/2)*(-N/2+M) - (1j*dr[1]/2)*(N/2))
        
    return r

In [3]:
def swap_st(N,Mj,Mk,ch):
    M = max(abs(Mj),abs(Mk))
    H = 0
    Mmax = max(Mj,Mk)
    Mmin = min(Mj,Mk)
    for J in np.arange(abs(M),N/2+1,1):
        if ch=='x':
            jmm1 = {(J,Mmax,Mmin):1,(J,Mmin,Mmax):1}
            coef = 1

        elif ch=='y':
            jmm1 = {(J,Mmax,Mmin):-1,(J,Mmin,Mmax):1}
            coef = 1j
        
        H += coef*dicke_basis(N,jmm1=jmm1)
        
    return (H)

def tau(N,J,M1,M2,j,m,ch,dr,dt):
    cm1 = abs(M2)/(abs(M1)+abs(M2))
    cm2 = abs(M1)/(abs(M1)+abs(M2))

    Xm2 = rate(ch,j,m,N,J,M2,dr,dt)
    Xp1 = rate(ch,j,m,N,J,M1,dr,dt)
    
    num = (cm2*Xm2 + cm1*Xp1)**2
    den = cm2*(Xm2)**2 + cm1*(Xp1)**2
    
    arg = np.sqrt(num/den)
    t = np.arccos(arg)
    
    return (t)

def sgn_tau(j,m,t0,t1):
    if m==1:
        if j==1:
            t0 = t0
            t1 = t1

        elif j==0:
            t0 = -t0
            t1 = t1

        else:
            t0 = -t0
            t1 = -t1

    elif m==0:
        if j==-1:
            t0 = -t0
            t1 = t1

        elif j==0:
            t0 = t0
            t1 = t1

        else:
            t0 = -t0
            t1 = t1

    else:
        if j==-1:
            t0 = t0
            t1 = t1

        elif j==0:
            t0 = -t0
            t1 = t1

        else:
            t0 = -t0
            t1 = -t1

    return (t0,t1)

def Recovery(ErSt,N,J,M1,M2,j,m,dr,dt):
    sSt = ErSt
    
    if abs(m)==1:
        H1 = swap_st(N,M1+m,M1,'x') + swap_st(N,M2+m,M2,'x') + swap_st(N,-M2+m,-M2,'x') + swap_st(N,-M1+m,-M1,'x')
        tend = np.pi/2
        
        t = np.linspace(0,tend,100)
        result = mesolve(H1,ErSt,t,[])
        sSt = result.states[-1]
    
    t1 = tau(N,J,M1,-M2,j,m,'c',dr,dt)
    t2 = tau(N,J,-M1,M2,j,m,'c',dr,dt)

    if j!=2:
        t1,t2 = sgn_tau(j,m,t1,t2)
    
    #print(f'{t1}\t{t2}')
    
    t = np.linspace(0,t2,100)
    result = mesolve(swap_st(N,-M1,M2,'y'),sSt,t,[])
    sSt = result.states[-1]
    
    t = np.linspace(0,t1,100)
    result = mesolve(swap_st(N,M1,-M2,'y'),sSt,t,[])
    sSt = result.states[-1]

    return (sSt)

In [4]:
def EncodedSt(M1, M2, α, β, ch = 1, N = 12, J = 5):
    if J<=abs(M1):
        J = abs(M1)
        
    j  = np.array([jj for jj in np.linspace(N/2,N%2/2,int(N//2+1))])
    mj = np.array([int(2*jj+1) for jj in j])
    dim = np.sum(mj)
    st_ind = np.sum(mj[:np.where(j==J)[0][0]])

    Log0 = np.zeros((dim,1),dtype=np.complex128)
    Log1 = np.zeros((dim,1),dtype=np.complex128)

    if ch:
        cm1 = np.sqrt(M2/(M1+M2))
        cm2 = np.sqrt(M1/(M1+M2))
    
        #Logical 0 state
        Log0[st_ind+J-M1] = α*cm1
        Log0[st_ind+J+M2] = α*cm2
    
        #Logical 1 state
        Log1[st_ind+J+M1] = β*cm1
        Log1[st_ind+J-M2] = β*cm2

    else:
        #Logical 0 state
        Log0[st_ind+J-(-J)] = α
    
        #Logical 1 state
        Log1[st_ind+J-(-J+1)] = β
        
    Log0 = Qobj(Log0)
    Log1 = Qobj(Log1)
    
    #Encoded state
    EncSt = Log0 + Log1

    return EncSt

In [5]:
## States after individual decay process
def dec_St(St,dr,N=12,J=5):
    j  = np.array([jj for jj in np.linspace(N/2,N%2/2,int(N//2+1))])
    mj = np.array([int(2*jj+1) for jj in j])
    dim = np.sum(mj)
    st_ind = np.sum(mj[:np.where(j==J)[0][0]])

    # Finding M levels occupied in the input state in J subspace
    i, _ = np.nonzero(St)
    mi = [st_ind+J-i[ind] for ind in range(len(i))]

    # Coefficients in individual decay
    
    Md1 = [bm(J,mi[ind]) for ind in range(len(mi))]
    Md2 = [am(J,mi[ind]) for ind in range(len(mi))]
    Md3 = [dm(J,mi[ind]) for ind in range(len(mi))]

    Me1 = [bp(J,mi[ind]) for ind in range(len(mi))]
    Me2 = [ap(J,mi[ind]) for ind in range(len(mi))]
    Me3 = [dp(J,mi[ind]) for ind in range(len(mi))]

    Mz1 = [bz(J,mi[ind]) for ind in range(len(mi))]
    Mz2 = [mi[ind] for ind in range(len(mi))]
    Mz3 = [dz(J,mi[ind]) for ind in range(len(mi))]
    
    # Finding possible J subspaces after decay and their coefficients
    if J==N/2:
        Jdec = [J-1,J,J-1,J,J-1,J]
        Cdec = [cjm1(N,J),cj(N,J),cjm1(N,J),cj(N,J),cjm1(N,J),cj(N,J)]
        Mdec = [Md1,Md2,Mz1,Mz2,Me1,Me2]
        mm = [0,0,1,1,2,2]

    elif J>0 and J<N/2:
        Jdec = [J-1,J,J+1,J-1,J,J+1,J-1,J,J+1]
        Cdec = [cjm1(N,J),cj(N,J),cjp1(N,J),cjm1(N,J),cj(N,J),cjp1(N,J),cjm1(N,J),cj(N,J),cjp1(N,J)]
        Mdec = [Md1,Md2,Md3,Mz1,Mz2,Mz3,Me1,Me2,Me3]
        mm = [0,0,0,1,1,1,2,2,2]

    elif J==0:
        Jdec = [J+1,J+1,J+1]
        Cdec = [cjp1(N,J),cjp1(N,J),cjp1(N,J)]
        Mdec = [Md3,Mz3,Me3]
        mm = [0,1,2]

    else:
        raise ValueError('Invalid J!')
        
    # Computation of States after decay
    St_dec = []
    St_Norm = []
    
    ii = 0
    for J_dec in Jdec:
        st_ind = np.sum(mj[:np.where(j==J_dec)[0][0]])
        st_ind0 = np.sum(mj[:np.where(j==J)[0][0]])
        st_dec = np.zeros((dim,1),dtype=np.complex128)
        for ind in range(len(mi)):
            if mi[ind]+(-1+mm[ii]) <=J_dec and mi[ind]+(-1+mm[ii]) >= -J_dec:
                st_dec[st_ind+J_dec-(mi[ind]+(-1+mm[ii]))] = Mdec[ii][ind]*St[st_ind0+J-(mi[ind])]
        
        nrm = np.linalg.norm(st_dec)
        St_dec.append(Qobj(st_dec)/nrm)
        St_Norm.append(dr[mm[ii]]*Cdec[ii]*nrm**2)
        ii += 1
        
    return (St_dec,np.array(St_Norm),Jdec,mm)

In [6]:
## Effective Hamiltonian:
def Hamiltonians(γ,N=12):
    system = Dicke(N = N)
    [Jx, Jy, Jz] = jspin(N)
    
    He = (-1j*γ[0]/2)*(-Jz+N/2) + (-1j*γ[2]/2)*(Jz-N/2) + (-1j*γ[1]/2)*(N/2)
    return He

## Monte-Carlo Evolution:
def Evolve(H,psi0,tmax,h,dr,ϵ1,ϵ2,N,J,M1,M2,α,β,ch1,ch2,ch):
    Ersti = []
    Ensti = []
    
    Ersti.append(psi0)
    Ensti.append(psi0)

    cnt,cnt1,cp1,cm1,cz = 0,0,0,0,0; Ji=J;j=0;m=-1;tnr=0;
    t = h; i = 0;
    while t<=tmax:
        decSt, decSt_nrm, decJ, Mm = dec_St(Ersti[-1],dr,N,J)
        dp = np.sum(decSt_nrm)
        dP = np.cumsum(decSt_nrm)/dp
        if h*dp < ϵ1[i] :
            st_d = (1-1j*h*H)*Ersti[-1]
            st_d = st_d/st_d.norm()
            if ch2:
                st_d = Ersti[-1]#Recovery(st_d,N,J,M1,M2,2,2,dr,h)
                #print(0,fidelity(st_d,EncodedSt(M1,M2,α,β,ch,N,J)))
        
        else:
            k = np.searchsorted(dP,ϵ2[i],side='left')
            st_d = decSt[k]
            J = decJ[k]
            m = -1+Mm[k]
            j = J-Ji
            cnt += 1;
            if m==1:
                cp1 += 1
            elif m==-1:
                cm1 += 1
            else:
                cz  += 1
            
            # Performing recovery
            if ch1:
                #print(J,Ji,j,m)
                if J>abs(M1):
                    st_d = EncodedSt(M1,M2,α,β,ch,N,J)
                    #st_d = Recovery(st_d,N,Ji,M1,M2,j,m,dr,h)
                    #print(J,Ji,j,m)
                    #print(1,fidelity(st_d,EncodedSt(M1,M2,α,β,ch,N,J)))
                    
                    cnt1 += 1;

                if J==abs(M1)+1 and ch1==2:
                    st_d = EncodedSt(M1,M2,α,β,ch,N,int(N/2))
                    J = int(N/2)
                    
                    if tnr==0:
                        tnr = t
        
        Ersti.append(st_d)
        Ensti.append(EncodedSt(M1,M2,α,β,ch,N,J))

        Ji = J
        t += h
        i += 1

    file = open('Data00/Data00.txt','a+')
    file.write(f'{ch}\t{ch2}\t{ch1}\t{cnt}\t{cnt1}\t{cm1}\t{cz}\t{cp1}\t{tnr}\n')
    file.close()
    
    return (Ersti,Ensti)

In [7]:
N = 20; J = 10;
M1 = 5; M2 = 2;
α = 0.707; β = np.sqrt(1-abs(α)**2);
tmax = 1; dr = [1,1,1]                             #decay rate

EnSt = EncodedSt(M1,M2,α,β,1,N,J)
UnEnSt = EncodedSt(M1,M2,α,β,0,N,J)

Hnp = Hamiltonians(dr,N)

h = 1/np.linalg.norm(Hnp, ord ='fro') # timestep
print('Time Step = ',h)

nmax = 2000; npts = int(tmax/h);
print('No of data points = ',npts)
print('No of trajectories = ',nmax)

t = np.linspace(0,tmax,npts+1)
print(EnSt.data)
print(UnEnSt.data)

Time Step =  0.01818181818181818
No of data points =  55
No of trajectories =  2000
  (5, 0)	(0.37790739606416807+0j)
  (8, 0)	(0.5977045376151283+0j)
  (12, 0)	(0.5975240580930612+0j)
  (15, 0)	(0.3780215413363182+0j)
  (19, 0)	(0.7072135462503529+0j)
  (20, 0)	(0.707+0j)


In [9]:
name = '00'
stt = ['Encoded','UnEncoded']
index = 0
ch0 = [1,0]
for EncSt in [EnSt]:
    print(f'Starting Calculations for {stt[index]} state')
    data = np.zeros((nmax,len(t)))
    dataec = np.zeros((nmax,len(t)))
    dataecc = np.zeros((nmax,len(t)))

    data1 = np.zeros((nmax,len(t)))
    dataec1 = np.zeros((nmax,len(t)))
    dataecc1 = np.zeros((nmax,len(t)))
    
    for ntraj in range(nmax):
        prob1 = np.random.uniform(0.0, 1.0, npts+1)    #Uniform random numbers
        prob2 = np.random.uniform(0.0, 1.0, npts+1)
        
        #print("\nCase1\n")
        ESt_nr, CSt_nr = Evolve(Hnp,EncSt,tmax,h,dr,prob1,prob2,N,J,M1,M2,α,β,0,0,ch0[index])
        EncFidnr = [(fidelity(ESt_nr[i],CSt_nr[i])) for i in range(len(ESt_nr))]
        data[ntraj,:len(EncFidnr)] = EncFidnr[:]

        #ESt_nr, CSt_nr = Evolve(Hnp,EncSt,tmax,h,dr,prob1,prob2,N,J,M1,M2,α,β,0,1,ch0[index])
        #EncFidnr = [(fidelity(ESt_nr[i],CSt_nr[i])) for i in range(len(ESt_nr))]
        #data1[ntraj,:] = EncFidnr[:]
        
        #if index==0:
            #print("\nCase2\n")
            #ESt_wr, CSt_wr= Evolve(Hnp,EncSt,tmax,h,dr,prob1,prob2,N,J,M1,M2,α,β,1,0,ch0[index])
            #EncFidwr = [(fidelity(ESt_wr[i],CSt_wr[i])) for i in range(len(ESt_wr))]
            #dataec[ntraj,:] = EncFidwr[:len(t)]

            #ESt_wr, CSt_wr= Evolve(Hnp,EncSt,tmax,h,dr,prob1,prob2,N,J,M1,M2,α,β,1,1,ch0[index])
            #EncFidwr = [(fidelity(ESt_wr[i],CSt_wr[i])) for i in range(len(ESt_wr))]
            #dataec1[ntraj,:] = EncFidwr[:len(t)]

        #if index==0:
            #print("\nCase3\n")
            #ESt_wrr, CSt_wrr= Evolve(Hnp,EncSt,tmax,h,dr,prob1,prob2,N,J,M1,M2,α,β,2,0,ch0[index])
            #EncFidwrr = [(fidelity(ESt_wrr[i],CSt_wrr[i])) for i in range(len(ESt_wrr))]
            #dataecc[ntraj,:] = EncFidwrr[:]

            #ESt_wrr, CSt_wrr= Evolve(Hnp,EncSt,tmax,h,dr,prob1,prob2,N,J,M1,M2,α,β,2,1,ch0[index])
            #EncFidwrr = [(fidelity(ESt_wrr[i],CSt_wrr[i])) for i in range(len(ESt_wrr))]
            #dataecc1[ntraj,:] = EncFidwrr[:]
    
        if ntraj%1000==999:
            print(f'Finished Calculations for {ntraj+1} trajectories.')

            nn = ntraj+1
            DataEn =    np.array([np.average(data[:nn,j]) for j in range(len(t))])
            DataEc =    np.array([np.average(dataec[:nn,j]) for j in range(len(t))])
            DataEcc =   np.array([np.average(dataecc[:nn,j]) for j in range(len(t))])
            DataEn1 = np.array([np.average(data1[:nn,j]) for j in range(len(t))])
            DataEc1 = np.array([np.average(dataec1[:nn,j]) for j in range(len(t))])
            DataEcc1= np.array([np.average(dataecc1[:nn,j]) for j in range(len(t))])
        
            file = open(f'Data{name}/Data{name}_Local{stt[index]}_{N}_{J}_{nn}.txt','a+')
            for i in range(len(DataEn)):
                file.write(f'{t[i]}\t{DataEn[i]}\t{DataEn1[i]}\t{DataEc[i]}\t{DataEc1[i]}\t{DataEcc[i]}\t{DataEcc1[i]}\n')
            file.close()

    index += 1
print(f'Finished Calculations.')

Starting Calculations for Encoded state


/home/harsh/.local/lib/python3.9/site-packages/scipy/sparse/base.py:583: RuntimeWarning: divide by zero encountered in double_scalars
  r = self._mul_scalar(1./other)


Finished Calculations for 1000 trajectories.
Finished Calculations for 2000 trajectories.
Finished Calculations.
